## 기업별 비형정 데이터 적재
### 기업별 네이버 뉴스 크롤링
* Kospi200 기업별 뉴스 크롤링.
    * 크롤링 기간: 약 2주
    * 추후 데이터 중복이 되지 않도록 범위 선정에 유의
* 뉴스 본문, 제목, 형태소 분석 결과를 news collection에 적재. (기업명, 기업코드 추가)
* selnuim, 사용
* chromedriver version 및 설치 경로 확인

In [1]:
!pip install pymongo
!pip install selenium
!pip install beautifulsoup4

In [6]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
from pymongo import MongoClient
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait

In [3]:
# chrome driver setting.
driver_path = '../Driver/MacOS/chromedriver'

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.122 Safari/537.36")

driver = webdriver.Chrome(driver_path, chrome_options=options)
driver.implicitly_wait(3)

/Users/hyoungrolee/anaconda3/envs/cs/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: use options instead of chrome_options
  # Remove the CWD from sys.path while we load stuff.


In [7]:
conn = MongoClient('192.168.0.12', 27017)
collection_category = conn['Changsung']['stock_category']
df = pd.DataFrame(list(collection_category.find({})))
df

,_id,company,code,rank,business_code,business
0,5f31fffb42a6826a9bc95d94,포스코케미칼,003670.KS,1,32302,"내화, 비내화 요업제품 제조업"
1,5f31fffb42a6826a9bc95d95,더블유게임즈,192080.KS,2,105802,소프트웨어 개발 및 공급업
2,5f31fffb42a6826a9bc95d96,우리금융지주,316140.KS,3,116409,기타 금융업
3,5f31fffb42a6826a9bc95d97,HDC현대산업개발,294870.KS,4,64101,건물 건설업
4,5f31fffb42a6826a9bc95d98,애경산업,018250.KS,5,32004,기타 화학제품 제조업
...,...,...,...,...,...,...
195,5f31fffb42a6826a9bc95e57,삼양홀딩스,000070.KS,196,116409,기타 금융업
196,5f31fffb42a6826a9bc95e58,한국테크놀로지그룹,000240.KS,197,116409,기타 금융업
197,5f31fffb42a6826a9bc95e59,대한항공,003490.KS,198,85101,항공 여객 운송업
198,5f31fffb42a6826a9bc95e5a,유한양행,000100.KS,199,32102,의약품 제조업


In [9]:
query_keywords = df['company'].unique()
s_date = '2020.06.01'
e_date = '2020.06.15'
s_from = s_date.replace(".","")
e_to = e_date.replace(".","")
len(query_keywords), query_keywords

(200,
 array(['포스코케미칼', '더블유게임즈', '우리금융지주', 'HDC현대산업개발', '애경산업', '셀트리온', '쿠쿠홈시스',
        'SK케미칼', 'BGF리테일', '카카오', '오리온', '넷마블', '현대중공업지주', '두산밥캣',
        '삼성바이오로직스', '화승엔터프라이즈', '동서', 'LIG넥스원', '이노션', '삼성물산', '삼성에스디에스',
        '만도', '쿠쿠홀딩스', 'BGF', '코스맥스', '종근당', '현대로템', '한진칼', '동아에스티',
        '한국콜마', '한국타이어앤테크놀로지', 'GS리테일', '삼양사', '신세계인터내셔날', '한국항공우주',
        '롯데하이마트', '이마트', 'BNK금융지주', '일진머티리얼즈', '현대위아', '휠라홀딩스', '현대홈쇼핑',
        '한미약품', '한라홀딩스', '삼성생명', '한화생명', '코오롱인더', '락앤락', '한전기술', 'GKL',
        'SK', '하이트진로', '영원무역', 'LG하우시스', '한세실업', 'NAVER', 'KB금융', '풍산',
        'LG이노텍', 'LG유플러스', '아시아나항공', '한전KPS', 'CJ제일제당', '팬오션', 'SK이노베이션',
        '삼성카드', '후성', 'LF', '아모레퍼시픽', '롯데관광개발', '롯데쇼핑', '현대글로비스', '하나금융지주',
        '금호타이어', 'CJ CGV', '유니드', 'GS', 'LG디스플레이', '대교', '기업은행', '강원랜드',
        '한국금융지주', '엔씨소프트', '현대백화점', '대웅제약', '휴켐스', '한샘', 'LG전자', 'S&T모티브',
        '신한지주', '코웨이', 'LG생활건강', 'LG화학', '대우건설', '포스코인터내셔널', '대우조선해양',
        '두산인프라코어', '동원F&B', '두산중공업', '한국가스공사', '케이티앤지

In [ ]:
conn = MongoClient('192.168.0.12', 27017)
collection_news = conn['Changsung']['news']


